# Pool Test Workers

Worker functions for pool tests. These must be in an importable
module for multiprocessing with spawn context.

In [ ]:
#|default_exp pool.workers

In [ ]:
#|export
from netrun.rpc.base import ChannelClosed

In [ ]:
#|export
def echo_worker(channel, worker_id):
    """Echo worker for pool testing."""
    try:
        while True:
            key, data = channel.recv()
            channel.send(f"echo:{key}", {"worker_id": worker_id, "data": data})
    except ChannelClosed:
        pass

In [ ]:
#|export
def compute_worker(channel, worker_id):
    """Compute worker for pool testing."""
    try:
        while True:
            key, data = channel.recv()
            if key == "square":
                channel.send("result", data * data)
            elif key == "double":
                channel.send("result", data * 2)
    except ChannelClosed:
        pass

In [ ]:
#|export
def pid_worker(channel, worker_id):
    """Worker that reports its process ID."""
    import os
    try:
        while True:
            key, data = channel.recv()
            channel.send("pid", {"worker_id": worker_id, "pid": os.getpid()})
    except ChannelClosed:
        pass

In [ ]:
#|export
def raising_worker(channel, worker_id):
    """Worker that raises an exception on 'raise' message."""
    try:
        while True:
            key, data = channel.recv()
            if key == "raise":
                raise ValueError(f"Intentional error: {data}")
            channel.send("ok", data)
    except ChannelClosed:
        pass

In [ ]:
#|export
def immediate_exit_worker(channel, worker_id):
    """Worker that exits immediately without processing."""
    return  # Exit immediately

In [ ]:
#|export
def slow_worker(channel, worker_id):
    """Worker that takes time to respond."""
    import time
    try:
        while True:
            key, data = channel.recv()
            time.sleep(0.1)
            channel.send("done", data)
    except ChannelClosed:
        pass

In [ ]:
#|export
def printing_worker(channel, worker_id):
    """Worker that prints to stdout and stderr for testing output capture."""
    import sys
    print(f"Worker {worker_id} starting")
    sys.stderr.write(f"Worker {worker_id} stderr\n")
    try:
        while True:
            key, data = channel.recv()
            print(f"Worker {worker_id} got: {key}={data}")
            channel.send(f"echo:{key}", {"worker_id": worker_id, "data": data})
    except ChannelClosed:
        print(f"Worker {worker_id} stopping")